In [8]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [11]:
SEASONS = list(range(2016,2024))

In [51]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [25]:
async def get_html(url, selector, sleep=5 , retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print (await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timewout error on {url}")
            continue
        else:
            break
    return html



In [33]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url,"#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)


In [34]:
for season in SEASONS:
    await scrape_season(season)

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
Timewout error on https://basketball-reference.com/leagues/NBA_2016_games-may.html
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-

In [59]:
standings_files = os.listdir(STANDINGS_DIR)
print(standings_files)

['NBA_2022_games-october.html', 'NBA_2021_games-june.html', 'NBA_2020_games-march.html', 'NBA_2020_games-september.html', 'NBA_2020_games-january.html', 'NBA_2020_games-august.html', 'NBA_2023_games-may.html', 'NBA_2019_games-april.html', 'NBA_2023_games-april.html', 'NBA_2022_games-may.html', 'NBA_2019_games-february.html', 'NBA_2018_games-february.html', 'NBA_2023_games-january.html', 'NBA_2016_games-april.html', 'NBA_2021_games-march.html', 'NBA_2018_games-january.html', 'NBA_2017_games-february.html', 'NBA_2016_games-february.html', 'NBA_2017_games-october.html', 'NBA_2018_games-april.html', 'NBA_2020_games-december.html', 'NBA_2019_games-october.html', 'NBA_2020_games-november.html', 'NBA_2021_games-may.html', 'NBA_2021_games-december.html', 'NBA_2022_games-april.html', 'NBA_2022_games-december.html', 'NBA_2023_games-november.html', 'NBA_2023_games-december.html', 'NBA_2017_games-april.html', 'NBA_2022_games-november.html', 'NBA_2016_games-january.html', 'NBA_2018_games-october.ht

In [74]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w") as f:
            f.write(html)


In [75]:


for f in standings_files:
    filepath= os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

Nets vs Bucks, October 19, 2021 | Basketball-Reference.com
Warriors vs Lakers, October 19, 2021 | Basketball-Reference.com
Pacers vs Hornets, October 20, 2021 | Basketball-Reference.com
Bulls vs Pistons, October 20, 2021 | Basketball-Reference.com
Celtics vs Knicks, October 20, 2021 | Basketball-Reference.com
Wizards vs Raptors, October 20, 2021 | Basketball-Reference.com
Cavaliers vs Grizzlies, October 20, 2021 | Basketball-Reference.com
Rockets vs Timberwolves, October 20, 2021 | Basketball-Reference.com
Timewout error on https://www.basketball-reference.com/boxscores/202110200NOP.html
76ers vs Pelicans, October 20, 2021 | Basketball-Reference.com
Magic vs Spurs, October 20, 2021 | Basketball-Reference.com
Thunder vs Jazz, October 20, 2021 | Basketball-Reference.com
Kings vs Trail Blazers, October 20, 2021 | Basketball-Reference.com
Nuggets vs Suns, October 20, 2021 | Basketball-Reference.com
Mavericks vs Hawks, October 21, 2021 | Basketball-Reference.com
Bucks vs Heat, October 21, 2